In [1]:
! pip3 install torch torchvision torchaudio
! pip install transformers
! pip install ffmpeg-python
! pip install Js2Py
! pip install -U openai-whisper

     |████████████████████████████████| 792 kB 1.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 31.7 MB/s eta 0:00:01
     |████████████████████████████████| 63.2 MB 36.6 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 24.0 MB 46.4 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 22.2 MB/s eta 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=728959773bcc95d9c8dfaa62d5bcc7ec0801b28a3c1a8c7e46d4cf738bf33418
  Stored in directory: /tmp/pip-ephem-wheel-cache-6u_6z72u/wheels/09/a2/b6/7de9e2f763d72cb5fd70e9623d5880cfdc7f4e0d17842c2fd5
  Created wheel for lit: filename=lit-16.0.0-py3-none-any.whl size=93583 sha256=9809b36aba6f9a7

In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import io
import ffmpeg
from IPython.display import HTML, Audio
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
from scipy.io.wavfile import write
from scipy.io import wavfile
import scipy.signal as sps
import torch
from js2py import eval_js

In [3]:

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
  
    process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav', ar=16000)
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)
  
    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr, output, riff

In [4]:
device = 0 if torch.cuda.is_available() else "cpu"
device = 'cpu'

In [5]:
MODEL_NAME = 'pierreguillou/whisper-medium-portuguese'

In [6]:
from transformers import pipeline
lang = "pt"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [7]:
audio, _, _ = get_audio()

JsException: ReferenceError: data is not defined

In [ ]:
prediction = pipe(audio, return_timestamps=True)["chunks"]